### Read the Tx data file. 

Points to be noted:

- The transaction dataset contains all data merged from OMS and ECOM dataset across 2020,2021 and 2022

1. For 2021 First touch channel information is not available so appears as NaN, but this is anyways not important for our analysis

2. We will create a column called sequence which will assign a logical sequence to each month+year combination.

3. We will also create a channel called Channel type which will indicate if the channel is Paid/Organic channel type

In [57]:
df_oms_ecom=pd.read_csv('Transaction_Data.csv')

df_oms_ecom.loc[df_oms_ecom['Year']==2021,'Year']=0 #Year 2021, year value is 0
df_oms_ecom.loc[df_oms_ecom['Year']==2022,'Year']=12 #Year 2022, year value is 12
df_oms_ecom.loc[df_oms_ecom['Year']==2023,'Year']=24 #Year 2023, year value is 24
df_oms_ecom['Seq']=df_oms_ecom['Month']+df_oms_ecom['Year']
df_oms_ecom.head()

#Add a column called Channel type to understand whether the channel type was Paid/Organic channel
Paid=['Paid Search','TCP','Affiliate','Referral','Paid Social','CLM','Display','Privilege','Store']
Organic=['Organic Search','Direct','Internal','Organic Social',]

df_oms_ecom['Channel_Type']=df_oms_ecom['First Touch Channel'].apply(lambda x:'Paid' if x in Paid
                                                                     else 'Organic' if x in Organic
                                                                    else 'Unknown')

#Since 2021 Marketing channel is not important
df_oms_ecom[(df_oms_ecom['Seq']>12) & (df_oms_ecom['Seq']<29)].head()

<IPython.core.display.Javascript object>

C:\Users\24057\AppData\Local\Temp\ipykernel_12044\1058869131.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_oms_ecom=pd.read_csv('Transaction_Data.csv')


,OrderNo,OrderDate,Month,Year,CustomerID,Revenue,First Touch Channel,Seq,Channel_Type
280691,TC300031659601,2022-08-15,8,12,0357efbd641164bde4a63e0ce8f8f2da,2562.43,TCP,20,Paid
280692,TC300031637824,2022-08-15,8,12,1188e68053b9c3d0f6fe2468f764f47a,1095.00,TCP,20,Paid
280693,TC300031673490,2022-08-15,8,12,66b712b3992906474367d349e7672902,799.00,TCP,20,Paid
280694,300031636886,2022-08-15,8,12,eb8912792390dd07164983b61e57f9f3,699.00,Paid Search,20,Paid
280695,300031642826,2022-08-15,8,12,88f89b619ef1e08ce0936e1617c29def,683.14,Referral,20,Paid


### Part 1 Find out  Newly acquired customers in Performace Window(January-May 2022)

Identify who are the new customers in Jan-May 2022. To do this 
1. we will look at the first purchase by a customer within the Performance window
2. We will look at the last purchase by these same customers in the Pre purchase window
3. We will find out the difference between 2 and 1
4. We will discard all customers whose difference is <=12

This way we get all customers who made their first purchase between Jan-May and did not make a purchase anytime in the last 12 months

In [58]:
#Sort the master dataset by sequence of Order date for each customer
df_oms_ecom.sort_values(by=['CustomerID','OrderDate'],ascending=True,inplace=True)

#Performance window
df_jan22_may22=df_oms_ecom[(df_oms_ecom['Seq']>=13) & (df_oms_ecom['Seq']<=17)]
temp=df_jan22_may22[['CustomerID']]

#Pre performance window
df_jan21_dec21=df_oms_ecom[(df_oms_ecom['Seq']>=1) & (df_oms_ecom['Seq']<=12)]
df_jan21_dec21=df_jan21_dec21.merge(temp,on='CustomerID')

#Find out each customers first purchase in the Performance window
#df_first=df_jan22_may22.groupby(by='CustomerID').first().reset_index()
df_perf=df_jan22_may22[['CustomerID','Seq']]
df_perf.rename(columns={'Seq':'First'},inplace=True)

#Find out each customers last purchase in the Pre-Performance window
#df_last=df_jan21_dec21.groupby(by='CustomerID').last().reset_index()
df_jan21_dec21=df_jan21_dec21[['CustomerID','Seq']]
df_jan21_dec21.rename(columns={'Seq':'Last'},inplace=True)

#Merge the 2  dataframes to get the Seq of the first and last purchase made by the customer
df_diff=df_jan21_dec21.merge(df_perf,on='CustomerID',how='right')

#How many customers are considered in Performance window
a=df_diff['CustomerID'].nunique()
print('Customers considered  in Performance window:',a)

df_diff['Last']=df_diff['Last'].fillna(0)

#Find the diff
df_diff['Diff']=df_diff['First']-df_diff['Last']


#If difference is 0 retain, if difference is >12 retain else drop
df_diff=df_diff[(df_diff['Diff']>12)]
b=df_diff['CustomerID'].nunique()
print('New customers in Performance window:',b)

print('%New customers of all customers in Performance Window:',b/a)


C:\Users\24057\AppData\Local\Temp\ipykernel_12044\211191099.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_perf.rename(columns={'Seq':'First'},inplace=True)


Customers considered  in Performance window: 261760
New customers in Performance window: 239363
%New customers of all customers in Performance Window: 0.9144368887530563


### Part 2: Filter the data to retain customers whose 1st transaction was Paid

Since we will be calculating the ROI of a customer acquired by a PAID channel we will need to ensure the first transaction made in the performance window is Paid channel driven

In [59]:
#Get all transactions done by customers filtered in Part 1
df_diff=df_diff.groupby(by='CustomerID').first().reset_index()
df_diff=df_diff[['CustomerID']]
df_jan22_may22=df_diff.merge(df_jan22_may22,on='CustomerID')

#Find out the first transaction made by each customer in Jan-May
df_jan22_may22.sort_values(by=['CustomerID','OrderDate'],ascending=True,inplace=True)
df_cust_first=df_jan22_may22.groupby(by='CustomerID').first().reset_index()

#Retain only those transactions where the channel type is Paid
df_cust_first_paid_tx=df_cust_first[df_cust_first['Channel_Type']=='Paid']

df_cust_first_paid_tx=df_cust_first_paid_tx[['CustomerID','OrderNo','OrderDate','Month','Year','First Touch Channel',
                                      'Channel_Type']]

print("New customers who made a Paid transaction in the performance window:",df_cust_first_paid_tx['CustomerID'].nunique())
print("% of new customers who made a Paid transaction:",
      df_cust_first_paid_tx['CustomerID'].nunique()/df_cust_first['CustomerID'].nunique())

df_cust_first_paid_tx.head(10)

New customers who made a Paid transaction in the performance window: 107163
% of new customers who made a Paid transaction: 0.44770077246692264


,CustomerID,OrderNo,OrderDate,Month,Year,First Touch Channel,Channel_Type
0,00001d84fb7190e6300d5a77984e6f3c,300025829739,2022-05-04,5,12,Paid Search,Paid
1,0000798fe5f3201cadf596363e660435,300022118848,2022-01-25,1,12,Affiliate,Paid
4,0000cf33c48c5d7973cdef3c474ed871,300023637078,2022-03-17,3,12,Paid Search,Paid
5,000141c7d0784e5c27f5bacb16e4821e,300025869289,2022-05-06,5,12,Affiliate,Paid
13,0002b23932a37e0ceab9099cd0e55696,300022075838,2022-01-24,1,12,Paid Search,Paid
14,0002b8a442c82d47e68b1a3552388f15,300025505098,2022-04-28,4,12,Paid Search,Paid
17,0002f570873115f4751d8db02811bcd1,300022523452,2022-02-09,2,12,Paid Search,Paid
22,00048f028e9c82b572563626d98d08aa,TC300026065406,2022-05-09,5,12,TCP,Paid
23,00049ab42e333dd915497be2d5937cde,TC300026818448,2022-05-21,5,12,TCP,Paid
24,0004dadde0b566e79cfe1b26e1f603c4,300024048804,2022-03-31,3,12,Paid Search,Paid


### Part 3: Find out all transactions made by the customers across lifetime(Index+Repeat cohorts)

In [60]:
#We will retain the Customer ID information from the df_cust_first_paid_tx table
df_cust_first_paid_tx=df_cust_first_paid_tx[['CustomerID','OrderNo']]
df_cust_first_paid_tx.rename(columns={'OrderNo':'FirstOrderNo'},inplace=True)
#Create the lookforward window such that all transactions in the lifetime are available
df_jan22_may23=df_oms_ecom[(df_oms_ecom['Seq']>12) & (df_oms_ecom['Seq']<=29)]


#Retain data for the filtered Customer IDs in Lifetime window
df_jan22_may23=df_jan22_may23.merge(df_cust_first_paid_tx,on='CustomerID')

#Additionally we will add a column called Flag to the frame so that we know which row is the first order
df_jan22_may23['Flag']=df_jan22_may23['OrderNo'] ==df_jan22_may23['FirstOrderNo']

df_jan22_may23['Flag']=df_jan22_may23['Flag'].apply(lambda x:0 if x==False
                             else 1 if x==True
                             else 'Unknown')

print('Transactions made by New customers across lifetime:',df_jan22_may23['OrderNo'].nunique())
df_jan22_may23.sample(5)

Transactions made by New customers across lifetime: 188663


,OrderNo,OrderDate,Month,Year,CustomerID,Revenue,First Touch Channel,Seq,Channel_Type,FirstOrderNo,Flag
165705,300026428986,2022-05-15,5,12,e0c637540ac0073fd37230e50db1f857,14990.00,Paid Search,17,Paid,300026428986,1
109412,300030624564,2022-07-31,7,12,94329bcb64d9cd590024cb793c12b485,2493.00,NaN,19,Unknown,300021909629,0
1574,300031625104,2022-08-15,8,12,02049027cf4a793fad8c10028047348a,499.00,Paid Search,20,Paid,300025469569,0
42667,TC300027227795,2022-05-29,5,12,3977c97952555f28ff7918af00939390,3998.00,Affiliate,17,Paid,TC300027227795,1
18316,300026056733,2022-05-09,5,12,186d92873193bcefb96fe52915f420d3,3599.10,Paid Search,17,Paid,300026056733,1


### Part 4: Retain only Paid transactions for Index cohort and Organic Purchases for Repeat cohorts

Now that we have the flags set for new and return purchase we will retain:

1. If flag=1 then Channel type should be Paid
2. If flag =0 then Channel type should be Organic


In [61]:
df_jan22_may23=df_jan22_may23[(df_jan22_may23['Flag']==1) 
                        | ((df_jan22_may23['Flag']==0)
                           & (df_jan22_may23['Channel_Type']=='Organic'))]


print("#Total transactions made by new customers when Repeat paid transactions removed",
      df_jan22_may23['OrderNo'].nunique())

df_jan22_may23.head()

#Total transactions made by new customers when Repeat paid transactions removed 123768


,OrderNo,OrderDate,Month,Year,CustomerID,Revenue,First Touch Channel,Seq,Channel_Type,FirstOrderNo,Flag
0,300025829739,2022-05-04,5,12,00001d84fb7190e6300d5a77984e6f3c,8999.10,Paid Search,17,Paid,300025829739,1
1,300022118848,2022-01-25,1,12,0000798fe5f3201cadf596363e660435,2074.00,Affiliate,13,Paid,300022118848,1
2,300023637078,2022-03-17,3,12,0000cf33c48c5d7973cdef3c474ed871,20490.00,Paid Search,15,Paid,300023637078,1
5,300025869289,2022-05-06,5,12,000141c7d0784e5c27f5bacb16e4821e,23490.00,Affiliate,17,Paid,300025869289,1
6,300022075838,2022-01-24,1,12,0002b23932a37e0ceab9099cd0e55696,3349.00,Paid Search,13,Paid,300022075838,1


### Now we have the dataset cleaned and ready to be used for Cohort creations

### Part 5:Aggregate transactions for each Index and Repeat cohorts

1. We will filter out rows which are the first paid transactions(flag set to 1 from prev step)
2. Rename the Month and Year for this dataset to Index month, year and the channel associated to this Tx
3. Merge this back with the orginal set such that for each tx row we have information regarding the index data as well
   as the running row data 
4. We will then aggregate the dataset by Index month,Index_year to find out what is the Revenue share for each of these     groups

In [73]:
#First we will rename the rows which designate 1st purchase as Index_month,year,Acquired channel and Channel type
df_index=df_jan22_may23[df_jan22_may23['Flag']==1]

df_index.rename(columns={'Month':'Index_month','Year':'Index_year','First Touch Channel':'Acquired Channel',
                        'Channel_Type':'Acquired Channel Type'},inplace=True)
df_index=df_index[['CustomerID','Index_month','Index_year','Acquired Channel','Acquired Channel Type']]
df_index


#Next we will merge this dataframe with the overalldataframe so that we have index month,year for each line of transaction
# We will call this dataframe cohort

df_cohort=df_index.merge(df_jan22_may23,on=['CustomerID'])
df_cohort.sample(5)

C:\Users\24057\AppData\Local\Temp\ipykernel_12044\3408964926.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_index.rename(columns={'Month':'Index_month','Year':'Index_year','First Touch Channel':'Acquired Channel',


,CustomerID,Index_month,Index_year,Acquired Channel,Acquired Channel Type,OrderNo,OrderDate,Month,Year,Revenue,First Touch Channel,Seq,Channel_Type,FirstOrderNo,Flag
4070,0852aa636b942049d55f9fe918e561e3,3,12,Display,Paid,300023660630,2022-03-28,3,12,3059.10,Display,15,Paid,300023660630,1
52026,6bd610cee1147cbe1bd496ec8d7b44db,4,12,Paid Search,Paid,300024859401,2022-04-13,4,12,1614.05,Paid Search,16,Paid,300024859401,1
97724,ca1b14c0bf2698a5626c16540600ae9c,2,12,Affiliate,Paid,300022813529,2022-02-17,2,12,22998.00,Affiliate,14,Paid,300022813529,1
55000,71f826d12998a0a5eef1a29c2cf25825,3,12,Paid Search,Paid,300041040072,2023-01-08,1,24,2699.00,Organic Search,25,Organic,300024202353,0
85625,b12556dba2f39a2eebabdff3b5eb038c,3,12,Affiliate,Paid,300037387521,2022-11-05,11,12,999.00,Organic Search,23,Organic,300022422401,0


In [74]:
#We will create a column called Index_Seq which will be the first month+year and Current_Seq for Current Month+Year
df_cohort['Index_Cohort']=df_cohort['Index_month']+ df_cohort['Index_year']
df_cohort['Current_Seq']=df_cohort['Month']+ df_cohort['Year']

#Create a column - Cohort which will contain the cohort number by subtracting the First month from current month
#Since we are doing this only for first 5 months of 2022 we are interested only in the first 5 index months
df_cohort=df_cohort[df_cohort['Index_Cohort']<=17]

#We will create a column called Cohort which will be the difference in the Current month and the index month
df_cohort['Return_Cohort']=df_cohort['Current_Seq']-df_cohort['Index_Cohort']
df_cohort=df_cohort[df_cohort['Return_Cohort']<=12]
df_cohort['Index_Cohort']=df_cohort['Index_Cohort']-12

#We will reatain ony columns which are relevant to our cohort calculation
df_cohort=df_cohort[['CustomerID','Index_Cohort','Return_Cohort','Revenue','OrderNo','Acquired Channel','Acquired Channel Type']]
df_cohort.head()

,CustomerID,Index_Cohort,Return_Cohort,Revenue,OrderNo,Acquired Channel,Acquired Channel Type
0,00001d84fb7190e6300d5a77984e6f3c,5,0,8999.10,300025829739,Paid Search,Paid
1,0000798fe5f3201cadf596363e660435,1,0,2074.00,300022118848,Affiliate,Paid
2,0000cf33c48c5d7973cdef3c474ed871,3,0,20490.00,300023637078,Paid Search,Paid
3,000141c7d0784e5c27f5bacb16e4821e,5,0,23490.00,300025869289,Affiliate,Paid
4,0002b23932a37e0ceab9099cd0e55696,1,0,3349.00,300022075838,Paid Search,Paid


Now, we have the following data about each customer:
What was the net revenue generated by all customers for a Particular Index_cohort,return_chort combination

### Part 6: Calculate CLV of each Index cohort

Next we need to find out

1. The net revenue generated by all customers for a Particular Index_cohort,return_chort combination

2. The total customers for a Index_cohort

If we divide 1 by 2 we can get the net CLV generated by each customer in each of the 5 Index_cohorts


In [76]:
#Enter a channel we will do this for
channel=input('Please enter channel (Affiliate/Paid Search etc) ')
df_cohort=df_cohort[df_cohort['Acquired Channel']==channel]

#Caluclate sum of Revenue by customers in Each Index and each cohort
df_cohort_1=df_cohort.groupby(by=['Index_Cohort','Return_Cohort']).sum().reset_index()

#Caluclate count of customers in Each Index and each cohort
df_cohort_2=df_cohort.groupby(by=['Index_Cohort','Return_Cohort'])['OrderNo'].count().reset_index()
df_cohort_3=df_cohort.groupby(by=['Index_Cohort','Return_Cohort'])['CustomerID'].nunique().reset_index()
df_cohort_2.rename(columns={'OrderNo':'Order_count'},inplace=True)
df_cohort_3.rename(columns={'CustomerID':'Customer_count'},inplace=True)

#create a Revenue per customer column which will give average value of a customer in each cohort
x=df_cohort_2.merge(df_cohort_3,on=['Index_Cohort','Return_Cohort'])
df_cohort=df_cohort_1.merge(x,on=['Index_Cohort','Return_Cohort'])
df_cohort['RevinCr']=round(df_cohort['Revenue']/10000000,2)


#Step 5: Pivot this dataframe where rows=Index_month, columns='Cohort',values='Count'
pd.options.display.float_format = '{:.2f}'.format
df_rev_pivot=df_cohort.pivot_table(index='Index_Cohort',columns='Return_Cohort',values='RevinCr')
df_cust_pivot=df_cohort.pivot_table(index='Index_Cohort',columns='Return_Cohort',values='Customer_count')

Please enter channel (Affiliate/Paid Search etc) Affiliate


C:\Users\24057\AppData\Local\Temp\ipykernel_12044\289128523.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_cohort_1=df_cohort.groupby(by=['Index_Cohort','Return_Cohort']).sum().reset_index()


<IPython.core.display.Javascript object>

In [77]:
df_rev_pivot

Return_Cohort,0,1,2,3,4,5,6,7,8,9,10,11,12
Index_Cohort,,,,,,,,,,,,,
1,3.61,0.08,0.13,0.47,2.97,0.16,0.04,0.01,0.03,0.03,0.01,0.01,0.03
2,4.35,0.16,0.05,0.96,0.17,0.02,0.02,0.04,0.07,0.01,0.00,0.01,0.02
3,8.38,0.06,0.63,0.08,0.08,0.04,0.06,0.01,0.04,0.01,0.02,0.01,0.02
4,4.51,0.45,0.04,0.04,0.03,0.05,0.03,0.01,0.02,0.00,0.01,0.03,0.02
5,11.41,0.25,0.07,0.04,0.10,0.08,0.03,0.04,0.15,0.01,0.06,0.05,0.05


In [78]:
df_cust_pivot

Return_Cohort,0,1,2,3,4,5,6,7,8,9,10,11,12
Index_Cohort,,,,,,,,,,,,,
1,1763,42,25,76,319,35,15,9,13,11,8,7,6
2,1792,39,16,114,48,12,7,16,26,10,4,9,7
3,3664,35,127,62,45,20,23,11,32,8,15,9,11
4,2086,98,28,32,21,16,24,9,17,9,14,20,15
5,5232,172,56,28,48,28,29,25,26,14,46,24,30


In [79]:
#CLV of each customer acquired in Jan 2022 via Paid Search

#Net revenue generated by each cohort
cohort_revenue=df_rev_pivot.sum(axis=1)*10000000

#Total customer acquired in each cohort
cohort_count=df_cust_pivot[0]

print('CLV for',channel,':')
print(cohort_revenue/ cohort_count)

CLV for Affiliate :
Index_Cohort
1   42994.90
2   32812.50
3   25764.19
4   25119.85
5   23585.63
dtype: float64
